# Vector Search

In [14]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [15]:

client = QdrantClient("http://host.docker.internal:6333") #connecting to local Qdrant instance

## Q1. Embedding the query

In [16]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [17]:
query = 'I just discovered the course. Can I join now?'

In [18]:
model = TextEmbedding(model_name=model_handle)
embeddings = list(model.embed([query]))

In [19]:
print(embeddings)

[array([-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,  3.92670571e-02,
       -1.41797115e-02, -4.68342854e-02,  2.69862822e-02,  3.49039060e-02,
        1.53419217e-03, -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
       -4.87239498e-02, -9.90923923e-02,  6.02579764e-02, -1.86105379e-02,
       -8.81040139e-03, -3.01994831e-02, -1.42541815e-02, -3.87884212e-02,
       -2.76125662e-02, -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
        8.37959894e-02, -8.87329606e-02, -7.30260625e-02,  5.92846802e-02,
        4.07805674e-02,  7.72684822e-02, -4.60029697e-02,  3.98426895e-02,
        1.74821510e-02,  8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
        4.76479896e-02,  1.76566196e-02, -3.62013627e-02, -3.53408062e-02,
       -5.92537995e-03,  2.97091202e-02,  8.05883752e-02,  1.07132722e-02,
       -5.38192519e-02, -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
        7.05763168e-03,  7.70159791e-03, -5.17482529e-02,  5.46955574e-02,
       -7.80289904e-02, 

In [20]:
print("Length of embedding: ", len(embeddings[0]))

Length of embedding:  512


In [21]:
print("Min value of embedding: ", min(embeddings[0]))

Min value of embedding:  -0.11726373885183883


## Q2. Cosine similarity with another vector

In [35]:
doc = 'Can I still join the course after the start date?'

In [36]:
doc_embeddings = list(model.embed([doc]))


In [37]:
print("doc Dot Product query:", doc_embeddings[0].dot(embeddings[0]))

doc Dot Product query: 0.9008528895674548


## Q3. Ranking by cosine

In [38]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [43]:
i = 0
for doc in documents:
    embed = list(model.embed([doc['text']]))
    print(f"{i}: {doc_embeddings[0].dot(embed[0])}")
    i+=1



0: 0.8106057216700708
1: 0.8498906698471509
2: 0.7991335457966962
3: 0.6813047116438659
4: 0.6739288060941504


## Q4. Ranking by cosine, version two

In [45]:
i = 0
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    embed = list(model.embed([full_text]))
    print(f"{i}: {doc_embeddings[0].dot(embed[0])}")
    i+=1

0: 0.9273252252988442
1: 0.8594847511716719
2: 0.8396602144619353
3: 0.76320298378397
4: 0.7604449265387344


## Q5. Selecting the embedding model

In [47]:
import json
for model in TextEmbedding.list_supported_models():
    if model["dim"] < EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

## Q6. Indexing with qdrant (2 points)

In [53]:
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

In [49]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [54]:
# Define the collection name
collection_name = "hw02"

In [55]:
# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [56]:
points = []
id = 0

for doc in course['documents']:
    full_text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=full_text, model=model_handle),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "question": doc['question']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [57]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
question = 'I just discovered the course. Can I join now?'

In [60]:
def search(collection_name, model_handle, query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [61]:
results = search(collection_name, model_handle, question)

In [62]:
print(results)

points=[ScoredPoint(id=4, version=0, score=0.83649504, payload={'text': 'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.', 'section': '+-General course questions', 'question': 'What if my answer is not exactly the same as the choices presented?'}, vector=None, shard_key=None, order_value=None)]
